In [1]:
import transformers

# This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible
# answers are allowed or not).
squad_v2 = False
model_checkpoint = "deepset/roberta-base-squad2"
batch_size = 16

In [26]:
import json

from datasets import Dataset

# Read COVID dataset

def load_json_file_to_dict(file_name):
    return json.load(open(file_name))
data_dict = load_json_file_to_dict("data/covid-qa/covid-qa-dev.json")



In [76]:
def reconstruct_data(data_dict):
    result_dict = {
        'id':[], 'title':[], 'context':[], 'question':[], 'answers':[]
    }
    for article in data_dict['data']:
        for paragraph in article['paragraphs']:
            for qa_pair in paragraph['qas']:
                for ans in qa_pair["answers"]:
                    result_dict["answers"].append({
                        'answer_start': [ans["answer_start"]],
                        'text': [ans["text"]]
                    })
                    result_dict["question"].append(qa_pair["question"])
                    result_dict["context"].append(paragraph["context"])
                    result_dict["title"].append(paragraph["document_id"])
                    result_dict["id"].append(qa_pair["id"])

    return result_dict.copy()


In [77]:
sample_dataset = Dataset.from_dict(reconstruct_data(data_dict))

In [68]:
sample_dataset

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 203
})

In [23]:
# TODO: modify this into covid dataset version

In [2]:
from datasets import load_dataset, load_metric

# load the dataset
datasets = load_dataset("squad_v2" if squad_v2 else "squad")

Reusing dataset squad (/Users/daohuei/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

In [35]:
datasets["validation"]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 10570
})

In [3]:
from transformers import AutoTokenizer

# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# tokenizer("What is your name?", "My name is Sylvain.")

In [4]:
# train data preprocessing

max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
pad_on_right = tokenizer.padding_side == "right"

def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [5]:
# data tokenization
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

Loading cached processed dataset at /Users/daohuei/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-eb7c13e373f33819.arrow
Loading cached processed dataset at /Users/daohuei/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-4e66715979a095a9.arrow


In [6]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

# load model
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [7]:
from transformers import default_data_collator

# get data collator
data_collator = default_data_collator

In [8]:
# setup trainer
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [107]:
# validation set preprocessing
def prepare_validation_features(examples):
    
    print(len(examples["question"]))
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]
    print(len(examples["question"]))

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    print(len(tokenized_examples["input_ids"]))
    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]
        
    return tokenized_examples

In [110]:
from datasets import Dataset
test_sample_set_covid = Dataset.from_dict(sample_dataset[:5])
test_sample_set = Dataset.from_dict(datasets["validation"][:5])

# validation set tokenization
validation_features = test_sample_set_covid.map(
    prepare_validation_features,
    batched=True,
    remove_columns=test_sample_set_covid.column_names
)

  0%|          | 0/1 [00:00<?, ?ba/s]

5
5
227


In [111]:
key = "answers"
test_sample_set[key], test_sample_set_covid[key]

([{'answer_start': [177, 177, 177],
   'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']},
  {'answer_start': [249, 249, 249],
   'text': ['Carolina Panthers', 'Carolina Panthers', 'Carolina Panthers']},
  {'answer_start': [403, 355, 355],
   'text': ['Santa Clara, California',
    "Levi's Stadium",
    "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."]},
  {'answer_start': [177, 177, 177],
   'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']},
  {'answer_start': [488, 488, 521], 'text': ['gold', 'gold', 'gold']}],
 [{'answer_start': [433], 'text': ['Gram positive, anaerobic bacterium']},
  {'answer_start': [1992],
   'text': ['adaptive strategy that enables bacteria to survive harsh environmental conditions for prolonged periods of time']},
  {'answer_start': [2243], 'text': ['Spo0A']},
  {'answer_start': [4755], 'text': ['toxins A and B']},
  {'answer_start': [629],
   'text': ['The capacities of nanopore sequencing for viral diagnos

In [112]:
test_sample_set_covid, test_sample_set

(Dataset({
     features: ['id', 'title', 'context', 'question', 'answers'],
     num_rows: 5
 }),
 Dataset({
     features: ['id', 'title', 'context', 'question', 'answers'],
     num_rows: 5
 }))

In [113]:
validation_features

Dataset({
    features: ['input_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 227
})

In [114]:
# inference on val set
raw_predictions = trainer.predict(validation_features)

The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 227
  Batch size = 16


In [115]:
raw_predictions

PredictionOutput(predictions=(array([[ 1.8830386 , -8.950881  , -9.310952  , ..., -9.187948  ,
        -9.270747  , -9.673088  ],
       [ 1.9403212 , -8.757857  , -9.298976  , ..., -4.2572184 ,
        -0.08869352, -8.93426   ],
       [ 2.7002494 , -8.733148  , -9.223859  , ..., -9.121831  ,
        -7.9471684 , -9.290447  ],
       ...,
       [ 1.6147189 , -7.3919144 , -8.720754  , ..., -9.015832  ,
        -7.604792  , -9.078127  ],
       [ 2.7188263 , -8.298267  , -8.861992  , ..., -8.799892  ,
        -8.272102  , -9.457897  ],
       [ 2.0730934 , -8.020707  , -8.766091  , ..., -9.438511  ,
        -9.438511  , -9.438511  ]], dtype=float32), array([[ 2.1263626, -8.772525 , -8.876476 , ..., -8.618605 , -6.676522 ,
        -8.161184 ],
       [ 2.522217 , -8.636038 , -8.86185  , ..., -7.5498385, -3.805926 ,
        -9.091347 ],
       [ 3.302677 , -8.502097 , -8.830866 , ..., -7.072866 , -8.367661 ,
        -9.031145 ],
       ...,
       [ 2.0747316, -8.72905  , -8.9300785, ...

In [116]:
import collections

import numpy as np
from tqdm.auto import tqdm

# postprocessing on predictions
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [117]:
# postprocessing
final_predictions = postprocess_qa_predictions(test_sample_set_covid, validation_features, raw_predictions.predictions)

Post-processing 5 example predictions split into 227 features.


  0%|          | 0/5 [00:00<?, ?it/s]

In [138]:
pred_dict = {k:v for k,v in final_predictions.items()}

In [139]:
def create_sample_gold(data_dict, pred_dict):
    gold_dict = {
        "data": []
    }
    
    ids = pred_dict.keys()
    
    for article in data_dict['data']:
        gold_article = {"paragraphs":[]}
        
        for paragraph in article['paragraphs']:
            gold_paragraph = paragraph.copy()
            gold_paragraph["qas"] = []
            for qa_pair in paragraph['qas']:
                if qa_pair["id"] in ids:
                    gold_paragraph["qas"].append(qa_pair)
            gold_article["paragraphs"].append(gold_paragraph)
        gold_dict["data"].append(gold_article)
    return gold_dict
gold_dict = create_sample_gold(data_dict, pred_dict)

In [140]:
# save to the file
def dict_to_json(data_dict, file_name):
    with open(file_name, "w") as outfile:
        json.dump(data_dict, outfile)

In [141]:
dict_to_json(gold_dict, "sample_gold.json")
dict_to_json(pred_dict, "sample_pred.json")